# **Problema: Simulación de Transporte de Neutrones con Métodos Cuasi-Monte Carlo (QMC)**

## **Descripción del Problema**
En ingeniería nuclear, un desafío crítico es calcular el **flujo neutrónico** en reactores nucleares para garantizar operaciones seguras y eficientes. El flujo neutrónico describe la distribución espacial de neutrones en el reactor y depende de:

1. **Geometría del reactor**: Combustible (donde ocurren fisiones) y moderador (que reduce la energía de los neutrones).
2. **Secciones eficaces**: Probabilidades de absorción/colisión de neutrones con materiales.
3. **Trayectorias estocásticas**: Movimiento aleatorio de neutrones en 3D (simplificado a 2D aquí).

Los métodos **Monte Carlo (MC)** tradicionales usan muestreo pseudoaleatorio, lo que puede generar ruido estadístico. Los métodos **cuasi-Monte Carlo (QMC)** con secuencias de baja discrepancia (Halton/Sobol) ofrecen una distribución más uniforme, reduciendo la varianza.

## **Código de Simulación**

In [1]:
import numpy as np
from scipy.stats import qmc
import matplotlib.pyplot as plt

# =============================================
# Configuración del reactor nuclear 2D
# =============================================
ancho_reactor = 100  # Tamaño del núcleo (cm)
radio_combustible = 10  # Radio de la barra de combustible (cm)
sigma_combustible = 0.1  # Sección eficaz de absorción en combustible [cm⁻¹]
sigma_moderador = 0.01  # Sección eficaz en moderador [cm⁻¹]

# =============================================
# Funciones auxiliares
# =============================================
def en_combustible(x, y):
    """Verifica si (x,y) está dentro del combustible."""
    centro = ancho_reactor / 2
    distancia = np.sqrt((x - centro)**2 + (y - centro)**2)
    return distancia < radio_combustible

def muestrear_direcciones(n, metodo='qmc'):
    """Muestrea direcciones de neutrones usando QMC o MC."""
    if metodo == 'qmc':
        muestreador = qmc.Halton(d=1, scramble=True)
        angulos = muestreador.random(n).flatten() * 2 * np.pi
    else:
        angulos = np.random.random(n) * 2 * np.pi
    return np.cos(angulos), np.sin(angulos)  # (dx, dy)

def simular_flujo(n_particulas, metodo='qmc'):
    """Simula el flujo neutrónico en 2D."""
    # Muestrear posiciones iniciales
    if metodo == 'qmc':
        muestreador = qmc.Halton(d=2, scramble=True)
        posiciones = muestreador.random(n_particulas) * ancho_reactor
    else:
        posiciones = np.random.random((n_particulas, 2)) * ancho_reactor
    
    dx, dy = muestrear_direcciones(n_particulas, metodo)
    flujo = np.zeros((ancho_reactor, ancho_reactor))
    
    for i in range(n_particulas):
        x, y = posiciones[i]
        while True:
            # Verificar límites del reactor
            if x < 0 or x >= ancho_reactor or y < 0 or y >= ancho_reactor:
                break
                
            # Determinar material
            if en_combustible(x, y):
                sigma = sigma_combustible
            else:
                sigma = sigma_moderador
                
            # Absorción neutrónica
            if np.random.random() < sigma:
                break
                
            # Registrar flujo
            x_int = int(np.clip(x, 0, ancho_reactor-1))
            y_int = int(np.clip(y, 0, ancho_reactor-1))
            flujo[x_int, y_int] += 1
            
            # Mover neutrón
            x += dx[i]
            y += dy[i]
    
    return flujo / n_particulas

# **Preguntas de Discusión**

1. **Análisis de resultados**:  
   - ¿Por qué el flujo neutrónico es más alto en el combustible que en el moderador?  
   - ¿Qué ventajas ofrece QMC en la visualización del gradiente de flujo?

2. **Métodos numéricos**:  
   - ¿Cómo afecta la dimensionalidad (2D vs 3D real) a la eficacia de QMC?  
   - ¿Por qué las secuencias de Halton/Sobol son más eficientes que el muestreo aleatorio en este problema?

3. **Ingeniería nuclear**:  
   - ¿Cómo impactaría un error alto en el cálculo de flujo en la seguridad de un reactor?  
   - ¿En qué otros problemas nucleares (ej: blindaje radiológico) sería útil QMC?

4. **Extensiones del código**:  
   - ¿Cómo modificarías el código para simular un reactor con múltiples barras de combustible?  
   - ¿Qué cambios se necesitan para incluir colisiones elásticas (dispersión de neutrones)?